In [1]:
from pyraf import iraf
iraf.task(casleoccd="casleoccd/casleoccd.cl")
iraf.task(lacos_im="casleoccd/lacos_im.cl")

iraf.noao()
iraf.imred()
iraf.ccdred()
iraf.imutil()
iraf.digiphot()
iraf.apphot()
iraf.obsutil()
iraf.stsdas()
# iraf.lacos_im()

imred/:
 argus/         ctioslit/       hydra/          kpnocoude/      vtel/
 bias/          dtoi/           iids/           kpnoslit/
 ccdred/        echelle/        irred/          quadred/
 crutil/        generic/        irs/            specred/
ccdred/:
 badpiximage    ccdlist         combine         mkillumcor      setinstrument
 ccdgroups      ccdmask         darkcombine     mkillumflat     zerocombine
 ccdhedit       ccdproc         flatcombine     mkskycor
 ccdinstrument  ccdtest/        mkfringecor     mkskyflat
digiphot/:
 apphot/        daophot/        photcal/        ptools/
apphot/:
 aptest         findpars@       pconvert        polymark        psort
 center         fitpsf          pdump           polypars@       qphot
 centerpars@    fitsky          pexamine        polyphot        radprof
 daofind        fitskypars@     phot            prenumber       wphot
 datapars@      pcalc           photpars@       pselect
obsutil/:
 bitcount       findgain        psfmeasure      

In [2]:
def casleoccd_hsh(image_list, band, images_path, base="./base_hsh/", master_bias=None, master_flat=None, master_dark=None, cosmic_correction=True):
    images_path = images_path+"/"
    instrumento = 'HSH-STL'  # Instrumento utilizado
    binning = 1  # Factor de binning (X,Y)
    redfoc = 'no'  # ¿Se usó reductor focal? (solo JS-Roper)
    ganron = 'no'  # ¿Calcula ganancia y ruido de lectura?
    sethead = 'yes'  # ¿Agrega/actualiza headers (EPADU y RDNOISE)?
    masterbias = 'yes' if master_bias is None else master_bias  # ¿Genera master BIAS? (Zero)
    masterflat = 'yes' if master_flat is None else master_flat  # ¿Genera master FLAT? (FlatX)
    masterdark = 'yes' if master_dark is None else master_dark  # ¿Genera master DARK? (Dark)
    borracal = 'no'  # ¿Borra calibraciones originales?
    procesa = 'yes'  # ¿Procesa todas las imágenes?
    ovscinter = 'no'  # ¿Ajusta overscan en forma interactiva? (solo JS Roper y Tek)
    oscuridad = 'yes'  # ¿Corrige por dark? (solo HSH-STL)
    cosmicos = 'yes' if cosmic_correction else 'no'  # ¿Corrige por rayos cósmicos (lacos-im)?
    epadu = 1.0  # Valor de ganancia calculado (e-/adu)
    rdnoise = 0.0  # Valor de ruido de lectura calculado (e-)

    iraf.casleoccd(instrumento=instrumento, binning=binning, redfoc=redfoc, ganron=ganron, sethead=sethead,
                   masterbias=masterbias, masterflat=masterflat, masterdark=masterdark, borracal=borracal,
                   procesa=procesa, ovscinter=ovscinter, oscuridad=oscuridad, cosmicos=cosmicos,
                   epadu=epadu, rdnoise=rdnoise, listbias=image_list['bias'], listflat=image_list['flat'+band],
                   listdark=image_list['dark'], listima=image_list['science'], flatcielo='no', flatHWP='no',
                   rfilJS=2, rfilHSH='ext', base=base, xcent=988, ycent=1072,
                  images_path=images_path)


def make_list(word, path):
    
    if word == "science":
        names = sorted([path+f for f in os.listdir(path) if (not "bias" in f.lower()) and 
                    (not "dark" in f.lower()) and (not "flat" in f.lower()) and 
                    (not "zero" in f.lower()) and (not "setpoint" in f.lower()) and 
                    (".fit" in f) ])
    else:
        names = sorted([path+f for f in os.listdir(path) if (word in f) and (".fit" in f)])
        
    with open(os.path.join(path, word + ".list"), 'w') as f:
        for name in names:
            f.write(name + '\n')
            
    return " ".join(names), os.path.join(path, word + ".list")

In [3]:
import os 

bands = "iv"
nights_path = "data_hsh/"
night = sorted(os.listdir(nights_path))[14]
night_path = os.path.join(nights_path, night)+"/"
night_images = os.listdir(night_path)
print(night_path)
image_list={}; kinds = ["bias", "dark", "science"]
for band in bands:
    kinds.append("flat"+band)
for kind in kinds:
    image_list[kind] = "@"+make_list(kind, night_path)[-1] #.split("/")[-1]


data_hsh/20240315/


In [4]:
image_list

{'bias': '@data_hsh/20240315/bias.list',
 'dark': '@data_hsh/20240315/dark.list',
 'science': '@data_hsh/20240315/science.list',
 'flati': '@data_hsh/20240315/flati.list',
 'flatv': '@data_hsh/20240315/flatv.list'}

In [10]:
casleoccd_hsh(image_list, band, night_path, master_bias=None, master_flat=None, master_dark=None, cosmic_correction=False)

 
Calcula ganancia (EPADU) y ruido de lectura (RDNOISE)


IrafError: Unknown parameter requested: "directory" for task: "cl" in pkg: "clpackage"

In [ ]:
import os

nights = os.listdir("nights_images")
filt = "i"

# Se generan los master bias, flat y dark si no existen previamente
# if not os.path.isfile('master_bias.fits'):
#     bias_list = [file for file in os.listdir(f"nights_images/{nights[0]}") if "bias" in file]
#     create_master_bias(bias_list)

# if not os.path.isfile('master_flat.fits'):
#     flat_list = [file for file in os.listdir(f"nights_images/{nights[0]}") if f"flat{filt}" in file]
#     create_master_flat(flat_list)

# if not os.path.isfile('master_dark.fits'):
#     dark_list = [file for file in os.listdir(f"nights_images/{nights[1]}") if "dark" in file]
#     create_master_dark(dark_list)
    

# Se generan las correcciones de rayos cósmicos si no existen previamente
if not os.path.isfile('cr_image1.fits'):
    lacos_im(['image1.fits'])
images_list = os.listdir(f"nights_images/{nights[2]}") 

casleoccd_hsh({'bias': 'master_bias.fits', 'flat': 'master_flat.fits', 'dark': 'master_dark.fits',
               'images': images_list}, cosmic_correction=False)


Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception
ERROR! Session/line number was not unique in database. History logging moved to new session 458


In [ ]:
" ".join(["a", "b"])

'a b'

In [ ]:
import os
nights = os.listdir("nights_images")
bias_list = [file for file in os.listdir(f"nights_images/{nights[0]}") if "bias" in file]
flat_i_list = [file for file in os.listdir(f"nights_images/{nights[0]}") if "flati" in file]
flat_v_list = [file for file in os.listdir(f"nights_images/{nights[0]}") if "flatv" in file]
dark_list = [file for file in os.listdir(f"nights_images/{nights[1]}") if "dark" in file]
os.listdir(f"nights_images/{nights[2]}") 

['CentaurusA_b0002.fit',
 'CentaurusA_b0007.fit',
 'CentaurusA_f0001.fit',
 'CentaurusA_f0006.fit',
 'CentaurusA_f0011.fit',
 'CentaurusA_f0012.fit',
 'CentaurusA_f0013.fit',
 'CentaurusA_f0014.fit',
 'CentaurusA_f0015.fit',
 'CentaurusA_f0016.fit',
 'CentaurusA_i0005.fit',
 'CentaurusA_i0010.fit',
 'CentaurusA_r0004.fit',
 'CentaurusA_r0009.fit',
 'CentaurusA_v0003.fit',
 'CentaurusA_v0008.fit']

In [ ]:
import os

def listar_directorios(path, nivel=0):
    for elemento in os.listdir(path):
        ruta_completa = os.path.join(path, elemento)
        if os.path.isdir(ruta_completa):
            print("|   " * nivel + "|-- " + elemento + "/")
            if "2024" in elemento:
                continue
            listar_directorios(ruta_completa, nivel + 1)
        else:
            print("|   " * nivel + "|-- " + elemento)

if __name__ == "__main__":
    directorio_base =  '../Observations_HSH'
    if os.path.isdir(directorio_base):
        print("Árbol de directorios para:", directorio_base)
        listar_directorios(directorio_base)
    else:
        print("El directorio especificado no existe.")


('\xc3\x81rbol de directorios para:', '../Observations_HSH')
|-- stsdas
|-- uparm/
|   |-- clecl.par
|-- data_reduction.py
|-- redu.py
|-- subsets/
|   |-- subsets_sbig-hsh_rext
|   |-- subsets_caspol
|   |-- subsets_sbig-hsh_rint
|   |-- subsets_roper_r2
|   |-- subsets_caspol-HWP
|   |-- subsets_roper_r1
|-- Settings/
|   |-- ccdtek-HWP.dat
|   |-- subsets_sbig-hsh_rext
|   |-- subsets_caspol
|   |-- subsets_sbig-hsh_rint
|   |-- ccdtek.dat
|   |-- ccdroper_r1.dat
|   |-- ccdsbig_hsh_rint.dat
|   |-- ccdsbig_hsh_rext.dat
|   |-- subsets_roper_r2
|   |-- subsets_caspol-HWP
|   |-- ccdroper_r2.dat
|   |-- subsets_roper_r1
|-- pyraf/
|   |-- clcache.v2/
|   |   |-- 0rwFKSevVndB54J0A+ivPg==
|   |   |-- V5A-jQWGB-i50zHVCLwi+Q==
|   |   |-- rA35OdKgMipLZ-qtNAUAbg==
|   |   |-- 1Vp-+GC6mmvzpgHCmNjA5g==
|   |   |-- aLo1NePVF8N8qDt5HC02xw==
|   |   |-- VU1ZH82P4r90jFXV0aAo6g==
|   |   |-- 7w5t990dmHyMvULHO7pi-A==
|   |   |-- urF7XXc0kB5SiMhRPc5zCg==
|   |   |-- vd5To6yC7aNdoCIY4Gah9A==
|   | 

In [ ]:
listar_directorios(".", nivel=0)

|-- stsdas
|-- uparm/
|   |-- clecl.par
|-- data_reduction.py
|-- redu.py
|-- subsets/
|   |-- subsets_sbig-hsh_rext
|   |-- subsets_caspol
|   |-- subsets_sbig-hsh_rint
|   |-- subsets_roper_r2
|   |-- subsets_caspol-HWP
|   |-- subsets_roper_r1
|-- Settings/
|   |-- ccdtek-HWP.dat
|   |-- subsets_sbig-hsh_rext
|   |-- subsets_caspol
|   |-- subsets_sbig-hsh_rint
|   |-- ccdtek.dat
|   |-- ccdroper_r1.dat
|   |-- ccdsbig_hsh_rint.dat
|   |-- ccdsbig_hsh_rext.dat
|   |-- subsets_roper_r2
|   |-- subsets_caspol-HWP
|   |-- ccdroper_r2.dat
|   |-- subsets_roper_r1
|-- pyraf/
|   |-- clcache.v2/
|   |   |-- 0rwFKSevVndB54J0A+ivPg==
|   |   |-- V5A-jQWGB-i50zHVCLwi+Q==
|   |   |-- rA35OdKgMipLZ-qtNAUAbg==
|   |   |-- 1Vp-+GC6mmvzpgHCmNjA5g==
|   |   |-- aLo1NePVF8N8qDt5HC02xw==
|   |   |-- VU1ZH82P4r90jFXV0aAo6g==
|   |   |-- 7w5t990dmHyMvULHO7pi-A==
|   |   |-- urF7XXc0kB5SiMhRPc5zCg==
|   |   |-- vd5To6yC7aNdoCIY4Gah9A==
|   |   |-- gHnB-9agvBoUT5ZpuQ52sg==
|   |   |-- 3lVIqYDK8prJ12m8Ge

In [ ]:
# Set up IRAF display parameters
# iraf.set(stdimage='imt1024', StdoutGraf='no')

def casleoccd_hsh(image_list, band, base="./base_hsh/", master_bias=None, master_flat=None, master_dark=None, cosmic_correction=True):
    instrumento = 'HSH-STL'  # Instrumento utilizado
    binning = 1  # Factor de binning (X,Y)
    redfoc = 'no'  # ¿Se usó reductor focal? (solo JS-Roper)
    ganron = 'no'  # ¿Calcula ganancia y ruido de lectura?
    sethead = 'no'  # ¿Agrega/actualiza headers (EPADU y RDNOISE)?
    masterbias = 'no' if master_bias is None else master_bias  # ¿Genera master BIAS? (Zero)
    masterflat = 'no' if master_flat is None else master_flat  # ¿Genera master FLAT? (FlatX)
    masterdark = 'no' if master_dark is None else master_dark  # ¿Genera master DARK? (Dark)
    borracal = 'no'  # ¿Borra calibraciones originales?
    procesa = 'yes'  # ¿Procesa todas las imágenes?
    ovscinter = 'no'  # ¿Ajusta overscan en forma interactiva? (solo JS Roper y Tek)
    oscuridad = 'yes'  # ¿Corrige por dark? (solo HSH-STL)
    cosmicos = 'yes' if cosmic_correction else 'no'  # ¿Corrige por rayos cósmicos (lacos-im)?
    epadu = 1.0  # Valor de ganancia calculado (e-/adu)
    rdnoise = 0.0  # Valor de ruido de lectura calculado (e-)
#     ccdsbig_hsh_rext_dat = './ccdsbig_hsh_rext.dat'  # Archivo de configuración específico del instrumento
#     subsets_sbig_hsh_rext = './subsets_sbig-hsh_rext'  # Archivo de subconjuntos de datos específicos del instrumento

    iraf.casleoccd(instrumento=instrumento, binning=binning, redfoc=redfoc, ganron=ganron, sethead=sethead,
                   masterbias=masterbias, masterflat=masterflat, masterdark=masterdark, borracal=borracal,
                   procesa=procesa, ovscinter=ovscinter, oscuridad=oscuridad, cosmicos=cosmicos,
                   epadu=epadu, rdnoise=rdnoise, listbias=image_list['bias'], listflat=image_list['flat'+band],
                   listdark=image_list['dark'], listima=image_list['science'], flatcielo='no', flatHWP='no',
                   rfilJS=2, rfilHSH='ext', base=base, xcent=988, ycent=1072)

# def create_master_bias(bias_list, output='master_bias.fits'):
#     master_bias = 'yes' if output else 'no'
#     iraf.casleoccd(instrumento='', binning=1, redfoc='', ganron='', sethead='', masterbias=master_bias,
#                    masterflat='', masterdark='', borracal='', procesa='', ovscinter='', oscuridad='',
#                    cosmicos='', epadu=1.0, rdnoise=0.0, listbias=bias_list, listflat='', listdark='',
#                    listima='', flatcielo='', flatHWP='', rfilJS=0, rfilHSH='', base='', xcent=0, ycent=0,
#                    intmt='', subst='', output=output, StdoutGraf='no')

# def create_master_flat(flat_list, output='master_flat.fits'):
#     master_flat = 'yes' if output else 'no'
#     iraf.casleoccd(instrumento='', binning=1, redfoc='', ganron='', sethead='', masterbias='',
#                    masterflat=master_flat, masterdark='', borracal='', procesa='', ovscinter='', oscuridad='',
#                    cosmicos='', epadu=1.0, rdnoise=0.0, listbias='', listflat=flat_list, listdark='',
#                    listima='', flatcielo='', flatHWP='', rfilJS=0, rfilHSH='', base='', xcent=0, ycent=0,
#                    intmt='', subst='', output=output, StdoutGraf='no')

# def create_master_dark(dark_list, output='master_dark.fits'):
#     master_dark = 'yes' if output else 'no'
#     iraf.casleoccd(instrumento='', binning=1, redfoc='', ganron='', sethead='', masterbias='',
#                    masterflat='', masterdark=master_dark, borracal='', procesa='', ovscinter='', oscuridad='',
#                    cosmicos='', epadu=1.0, rdnoise=0.0, listbias='', listflat='', listdark=dark_list,
#                    listima='', flatcielo='', flatHWP='', rfilJS=0, rfilHSH='', base='', xcent=0, ycent=0,
#                    intmt='', subst='', output=output, StdoutGraf='no')

# def lacos_im(image_list):
#     # Llama a la tarea de IRAF lacos_im para la corrección de rayos cósmicos
#     iraf.lacos_im(images=image_list, output='cr_', sigclip=4.5, sigfrac=0.3, objlim=1.0, gain=1.0, readn=6.5,
#                   overwrite=True, interactive=False, verbose=True, StdoutGraf='no')

def make_list(word, path, neg = False):
    
    if word is "science":
        names = sorted([f for f in os.listdir(path) if (not "bias" in f.lower()) and 
                    (not "dark" in f.lower()) and (not "flat" in f.lower()) and 
                    (not "zero" in f.lower()) and (not "setpoint" in f.lower()) and 
                    (".fit" in f) ])
    else:
        names = sorted([f for f in os.listdir(path) if (word in f) and (".fit" in f)])
        
    with open(os.path.join(path, word + ".list"), 'w') as f:
        for name in names:
            f.write(name + '\n')
            
    return os.path.join(path, word + ".list")